<a href="https://colab.research.google.com/github/Didgety/Didgety/blob/main/QuantumGenerativeModels_I_BornMachines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantum Generative Models I: The Quantum Circuit Born Machine**
QSC Summer School Elective Day 1: Training quantum models on classical data
\
Author:
* Kathleen E. Hamilton, ORNL, (hamiltonke@ornl.gov)


## Overview
The field of quantum machine learning (QML) is rapidly growing, and every day researchers around the world are constructing and publishing on models for:
* Supervised learning on classical data (classical features, classical labels, quantum model)
* Supervised learning on quantum data (quantum featuers, quantum labels, quantum model)
* Unsupervised learning
* Semi-supervised learning
* Discriminative modeling
* Causal modeling
* Probabilistic modeling
* **Generative modeling**

In addition to the plethora of learning tasks, there is so much data to work with and analyze.  

In additon to all the learning tasks, associated with each is a broad design space, with many choices available for data encoding, parameterized model construction and extracting information from quantum states.

I think this is all great, and the research landscape is ever growing.
 However, this elective course meets for 2 days and we have 2 hours each day.  My goal is to do the minimum amount of talking, and instead have the majority of our time dedicated to hands-on numerical coding and testing.

To that end, I have designed these Colab notebooks to act as a "Choose Your Own Adventure" -- there are $3$ Sections planned out as follows:

1. Training a generator on discrete value data
2. Training a generator on continuous value data
3. Training a image denoiser on discrete data


Under Sections 1--2 I have provided minimal working examples (MWEs) that rely on many pre-defined functions available in [PennyLane](https://pennylane.ai/).

Certain cost functions, examples of numerical datasets and reasonable guesses for hyperparameters are given.  The code in the MWEs should be executable without any modifications.

Underneath each MWE there are several questions and options to explore and build upon the MWE -- testing out different cost functions, building your own trainable circuit structure, trying other datasets.  

My goal for today is to walk through the MWEs, then you have the option to explore the one that most interests you.  Section 3 requires the most coding, it does not have an immediate MWE. While we will start discussing it during our class time, it may require additional time and is provided as a "homework" example, although please note, these are not required.

##Background

The purpose of this notebook is to work through the construction, training, and evaluation of generative models.  The purpose of this generative model is to produde data examples that are similar to those generated by an unknown stochastic process.  

We will mostly consider models that are trained on _unlabeled_ data.  The training dataset $\mathbf{X}=\lbrace(x_n)\rbrace$ consists of $M$-samples of m-length feature vectors $x_i$.  

###Classical Models

There are many examples of classical generative models: Boltzmann machine, generative adversarial networks, variational autoencoders, flows and diffusion models.  

### The Quantum Circuit Born Machine

The first quantum generative model we will use is the Quantum Circuit Born Machine (QCBM) [1].  This is a variational model that is trained to fit the marginal distribution of a fixed traininig dataset: $P(\mathbf{X})$.

For the Quantum Circuit Born Machine (QCBM) the datum $\mathbf{x}$ are binary strings of length $m$, where $m$ is the number of qubits in the circuit model.


#### References
[1]Benedetti, Marcello, et al. "A generative modeling approach for benchmarking and training shallow quantum circuits." npj Quantum Information 5.1 (2019): 45. https://arxiv.org/abs/1801.07686





#### SOLUTIONS
I have a roster of emails for everyone enrolled in this elective.  This QML elective only meets during Week 1 of the Summer School.  I will provide versions of this Colab that has my implementations between Week 1 and Week 2.

I hope this elective will be interesting, and that you enjoy these Colabs.  Any questions, concerns, or if you want to discuss interesting use cases of QML please reach out via my email.

In [ ]:
pip install scikit-learn

In [ ]:
pip install pennylane

###Class structure
The basic QCBM class is used for building a specific parameterized circuit of depth (L), projecting all qubits onto a fixed basis, and using gradient descent to train all parameters in the circuit.

The following MWE class definition was pulled from several codebases that I've developed -- inside the class there are several pre-defined functions:
* `build_circuit` builds the QCBM circuit with the specified number of layers, and layer type. The examples use the pre-defined templates found in Pennylane, but there is also an option to define a unique user template.
* `pdf` returns the distribution over the computational basis states.
* `spin_vector` returns the average spin of each qubit in the register $[\langle Z_0\rangle, \langle Z_1\rangle,\dots, \langle Z_{n-1}\rangle]$ (*not impelemented, needs to be added*
* `fit` trains the QCBM to return the optimal `pdf` with a specific cost function and optimizer.

While the majority of exercises today will use noiseless simulation, there is the option to test out using PennyLane's `mixed.default` simulator to test the effects of noise.  


In [ ]:
from pennylane import numpy as np
import pennylane as qml

import itertools as it
import collections
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pennylane.templates import BasicEntanglerLayers,StronglyEntanglingLayers,SimplifiedTwoDesign

# Stage 1: the MWE of a QCBM

In [ ]:
class QCBM:
    '''
    Parent class for constructing
    Quantum Circuit Born Machine, that uses PennyLane
    Inputs:
    wires -- number of qubits to build in the circuit
    shots -- number of shots to take from the prepared state
    layers -- how many layers of a template to add to the circuit (using a template)
    template -- which predefined template to use:
      'basic_template' = BasicEntanglerLayers,
      'strong_tempalte' = StronglyEntanglingLayers,
      'two_design_template' = SimplifiedTwoDesign,
      'user_defined_template' = user defined function
    backend -- 'sim' or 'noisy_sim' for simulator or noisy simulator
    **kwargs -- additional parameters for the class
    '''


    def __init__(self, wires=1,shots=1024,layers=1,backend='sim',**kwargs):
        self.dev_wires = [np.array(idx, requires_grad=True) for idx in range(wires)] # the individual device wires
        self.wires=wires # number of qubits in the circuit
        self.rotation = kwargs.get('rotation',None) #option for the BasicEntanglingLayer template which uses single axis rotations as the parameterized gate
        self.layers = layers # number of layers of template to add to the circuit
        self.layout = kwargs.get('layout','basic_template') # which template to use
        self.add_final_layer=kwargs.get('final_layer',False) # option for adding a final layer of rotations to the circuit
        self.backend=backend # options: 'sim' or 'noisy_sim'
        self.shots = shots # number of shots to sample from the final state
        self.n_iter_no_change=kwargs.get('wait_time',5) # hyperparameter for training that triggers early stopping
        self.tol = kwargs.get('tol',1e-5) # hyper parameter for training that triggers early stopping
        self.pBF=kwargs.get('pBF',None)  # option for noisy simulation controlling the strength of BitFlip errors prior to measurement
        self.pDP=kwargs.get('pDP',None) # option for noisy simulation controlling the strength of Depolarizing noise channel
        self.pAD=kwargs.get('pAD',None) # option for noisy simulation controlling the strength of Amplitude Damping noise channel
        self.pCE=kwargs.get('pCE',None) # option for noisy simulation controlling the strength of coherent errors
        self.prob_vec = True
        if self.backend=='sim':
            self.device = qml.device("default.qubit", \
                                     wires=self.dev_wires,shots=shots)
        elif self.backend=='noisy_sim':
            if (self.pDP!=None) or (self.pBF!=None)or (self.pAD!=None):
                ##print('you need a noisy device')
                self.device = qml.device('default.mixed',\
                                    wires = self.dev_wires,shots=shots)
        self.rng=np.random.default_rng(2024)

    def write_metadata(self):
      """Option: Add this functionality"""
      """ Add code here to initialize a directory to store output files"""
      """ Example: trained model parameters, loss curves, information about hyperparameters used"""
      raise NotImplementedError

    def build_circuit(self,params):
      """ Option: Add some code to check that the correct number of parameters
      have been passed to build the circuit """
      # # #
      # Here
      # # #
      if self.add_final_layer:
        unitary_params = params[:-self.wires*2]
        final_params = params[-self.wires*2:]
      else:
        unitary_params = params
        final_params = None
      """ Option: The MWE just uses the all zero state as the initial state """
      # # #
      # Add code here to provide the option to use a different initial state
      # # #
      qml.BasisState(np.zeros(self.wires), wires=self.dev_wires)
      if self.layout=='basic_template':
        shape = BasicEntanglerLayers.shape(n_layers=self.layers,\
                                                      n_wires=self.wires)
        unitary_params=np.asarray(unitary_params).reshape(shape)
        BasicEntanglerLayers(unitary_params,rotation=self.rotation,\
                                                        wires = self.dev_wires)
      elif self.layout=='strong_template':
        shape = StronglyEntanglingLayers.shape(n_layers=self.layers,\
                                                        n_wires=self.wires)
        unitary_params=np.asarray(unitary_params).reshape(shape)
        StronglyEntanglingLayers(unitary_params, wires = self.dev_wires)
      elif self.layout=='two_design_template':
        shape = SimplifiedTwoDesign.shape(n_layers=self.layers, \
                                                        n_wires=self.wires)
        unitary_params=np.asarray(unitary_params).reshape(shape)
        SimplifiedTwoDesign(unitary_params, wires = self.dev_wires)
      elif self.layout=='user_defined_template':
        # ADD YOUR CODE HERE
        # # #
        # write a function that can be passed to qml.template or use other
        # functionalitiy such as qml.broadcast to build your own parameterized
        # circuit design
        # # #
        raise NotImplementedError
      else:
        raise ValueError('Please choose a valid layout')
      if final_params is not None:
        # ADD YOUR CODE HERE
        # Ensure that final_params has the correct shape
        #
        qml.broadcast(unitary=qml.RY, pattern="single", \
                            wires=self.dev_wires, parameters=final_params[0])
        qml.broadcast(unitary=qml.RX, pattern="single", \
                      wires=self.dev_wires, parameters=final_params[1])

      if self.prob_vec:
          if self.pBF!=None:
              for idx in self.dev_wires:
                  qml.BitFlip(self.pBF, wires=self.dev_wires[int(idx)])
              return qml.probs(wires=self.dev_wires)
          return qml.probs(wires=self.dev_wires)
      else:
          return [qml.expval(qml.PauliZ(i)) for i in self.dev_wires]
    def draw_circuit(self,*args, **kwds):
        raise NotImplementedError

    def pdf(self,params):
        '''
        pennylane returns the probabilities in an OrderedDict - no need for sorting the values
        '''
        self.prob_vec=True
        qnode_ = qml.QNode(self.build_circuit, self.device)
        Q = qnode_(params)
        return Q

    def spin_vector(self,*args, **kwds):
      """ ADD CODE HERE """
      # Add code so that the output of the QCBM is a vector of [<Z0>, <Z1>,...]
      raise NotImplementedError

    def sample(self,*args,**kwds):
      """ ADD CODE HERE"""
      # Add code to prepare the pdf and draw N samples
      # return samples as bitstrings s
      raise NotImplementedError

    def fit(self,startval,opt_method,cost_fcn,target,nsteps=10,alpha=0.05):
      """ ADD CODE HERE """
      # Add code to handle the case that an argument is missing

      opt = opt_method(stepsize=alpha)

      params = startval.copy()

      loss = {}
      loss[0]=cost_fcn(target,self.pdf(params))
      print('initial loss: ',loss[0])

      stored_params = {}
      stored_params[0]=params.copy()

      for i in range(nsteps):
          # Update the circuit parameters
          params,_loss = opt.step_and_cost(lambda v: cost_fcn(target,self.pdf(v)), params)
          # Option: Does this print out too much during training?  add a verbose keyword to toggle that on/off
          if i%2==0:
              print(i+1,_loss)
          loss[i+1]=_loss.copy()
          stored_params[i+1]=params.copy()
      return params,loss,stored_params



Check out the MWE

In [ ]:
n_wires = 2
n_layers = 2
n_shots = 256
my_qcbm = QCBM(wires=n_wires,layers=n_layers,shots=n_shots,backend='sim')

In [ ]:
my_qcbm.__dict__

In [ ]:
thetas = np.random.uniform(size=(n_layers,n_wires))

In [ ]:
temp_qnode = qml.QNode(my_qcbm.build_circuit, my_qcbm.device)
qml.draw_mpl(temp_qnode,decimals=3,expansion_strategy='device')(thetas)


In [ ]:
my_qcbm.pdf(thetas)

In [ ]:
f, ax = plt.subplots(figsize = (10,6))
sns.barplot(my_qcbm.pdf(thetas),legend=False,ax=ax)
# option:  adjust the Matplotlib axes objects so that the x labels are bitstrings '00', '01', '10', '11'

## Density Modeling

In the first example, we are going to train the MWE QCBM to fit the probability density function of a dataset composed of one-dimensional features.  

In the MWE built on 2 wires, the output from `pdf(thetas)` is a distribution over $2^n =4$ binary bitstrings.  

Let's try to train this to prepare a specific distribution

In [ ]:
#Options for targets (choose one, comment out the rest)
default_rng = np.random.default_rng(2024) # random number generator
target = np.asarray([0, 0, 1, 0]) #prepare a specific bitstring with probability 100
#target = [0.5, 0, 0, 0.5] # prepare two bitstrings, equally probable
#target = [0.5, 0.5, 0, 0] # prepare two other bitstrings, equal probability
#target = [0, 0.25,0.75,0] # prepare two bitstrings, unequally
#target = default_rng.random(size=4) # random target -  CAUTION!! make sure this is a normalized vector before using it!!

Now we need a cost function.  I am choosing to use a cost function that will be minimized by gradient descent, so an ideal solution will be the set of parameters
$$\widehat{\theta} = \mathrm{argmin}_{\theta}(\ell(\theta))$$.

 Here are some examples, the Kullback-Leibler Divergence,  or the cosine similarity.

The KLD quantifies the similarity between two distributions $\mathbb{P}, \mathbb{Q}_{\theta}$ where $\mathbb{P}$, is the fixed target chosen above and $\mathbb{Q}_{\theta}$ is the distribution produced by the QCBM when `pdf(theta)` is called.

The KLD is defined as,
$$ \ell(\theta,\mathbb{P}) = KLD(\mathbb{P},\mathbb{Q}_{\theta}) = \sum_{i} p(x_i)\log{\frac{p(x_i)}{q_{\theta}(x_i)}}$$
where an overall constant term (the Shannon entropy of the target) was omitted and the summation is over the individual bitstring probabilities $p(x_i)$ and $q(x_i)$.

The cosine similarity is more general, and compute the similarity between two vectors $\mathbb{P}, \mathbb{Q}_{\theta}$ but does not require that the two vectors are normalized, but it does require that they are _normalizable_ (i.e. $|\mathbb{P}| \neq 0$ and $|\mathbb{Q}|\neq 0$).

$$ \ell(\theta,\mathbb{P}) = 1 - \frac{\mathbb{P}\cdot \mathbb{Q}_{\theta}}{|\mathbb{P}||\mathbb{Q}_{\theta}|} $$

In [ ]:
def kl_div(p,q):
    # if q = 0 that is going to cause problems
    # add a small epsilon to q to avoid np.nan or np.inf results
    from itertools import combinations
    qk = np.asarray(q)
    ck = np.broadcast(p, q)

    vec = [u*np.log(u/v) if (u>0 and v>0) else 0 if (u == 0 and v>=0) else np.log(1e10) for (u,v) in ck]
    S = np.sum(vec)
    return S

def cosine_distance(p,q):
    #p = target
    pnorm = np.sqrt(np.sum([x**2 for x in p]))

    #q = pdf(params)
    qnorm = np.sqrt(np.sum([x**2 for x in q]))
    return 1.0 - (p@q)/(pnorm*qnorm)

In [ ]:
trained_model,loss_curve,all_params = my_qcbm.fit(thetas,qml.AdamOptimizer,cosine_distance,target,nsteps=64)

the function `.fit(...)` returns 3 outputs:  the parameters of the final trained parameter, a dictionary storing the loss at each optimization step, and a dictionary all the parameters at each optimization step

In [ ]:
output = pd.DataFrame(my_qcbm.pdf(thetas))
output.columns = ['initial']
output['fitted'] = my_qcbm.pdf(trained_model)
output['target'] = target
f, ax = plt.subplots(figsize = (10,6))
output.plot(kind='bar',legend=True,ax=ax,rot=0)

In [ ]:
sns.lineplot(loss_curve.values(),legend=False)

# Section 1: Training a QCBM on discrete data

This MWE showed how a 2 qubit QCBM with a small number of parameters can fit a target density funcion.  Let's review the main componenets of the QCBM:



1.   The initial state is fixed, in the example above it was always the $|0\rangle^{\otimes n}$ state.   The `build_circuit()` function can be modified to take a different input state (ex: $|+\rangle^{\otimes n}$ or a binary state)
2.   When `pdf()` is called, the parameterized unitary is applied to the initial state and the prepared state is then projected onto a fixed basis
3.  From this distributions, you can draw individual samples


In this Section we look at a larger example (4 qubits) using and a common benchmark dataset known as the Bars and Stripes image set.

The Bars and Stripes image set is comprised of black and white images that depict either "bars" (columns are the same color all black or all white), or "stripes" (rows are the same color all black or all white).

For this elective the images are restricted to be square with $n$ pixels per side-- $(n \times n)$ total pixels.  Producing valid images requires $(n \times n)$ qubits.

The goal of this Section is to train a generative model to return data from a set defined from a fixed set of rules.



In [ ]:
def bars_and_stripes(n):
  ''' generate bars and stripes images of size n x n and return as binary strings'''
  output = np.zeros(2**(n**2))

  output[0]=1 #all zero bitstring is valid image
  output[-1]=1 # all one bitstring is valid image

  #generate all valid single stripe and single bar images
  for idx in range(n):
    temp = np.zeros((n,n),dtype=int)
    temp[idx]=1
    res = 0
    for ele in temp.ravel():
        res = (res << 1) | ele
    output[res]=1
    temp = np.zeros((n,n),dtype=int)
    temp.T[idx]=1
    res = 0
    for ele in temp.ravel():
        res = (res << 1) | ele
    output[res]=1
  for idx in range(2,n):
    for subset in it.combinations(list(range(n)), idx):
      temp = np.zeros((n,n),dtype=int)
      temp[list(subset)]=1
      res = 0
      for ele in temp.ravel():
          res = (res << 1) | ele
      output[res]=1
      temp = np.zeros((n,n),dtype=int)
      temp.T[list(subset)]=1
      res = 0
      for ele in temp.ravel():
          res = (res << 1) | ele
      output[res]=1
  return output


In [ ]:
def plot_bars_and_stripes(n):

  valid_images = np.where(bars_and_stripes(n)>0)[0]


  w = len(valid_images)//4
  h = 4
  fig, axs = plt.subplots(nrows=h, ncols=w,figsize=(10,10))

  for idx, ax in enumerate(axs.flat):
    ax.imshow(np.array([int(x) for x in np.binary_repr(valid_images[idx],width=n**2)]).reshape((n,n)).numpy())
    # Major ticks
    ax.set_xticks(np.arange(0, n, 1))
    ax.set_yticks(np.arange(0, n, 1))

    # Labels for major ticks
    #ax.set_xticklabels(np.arange(1, n+1, 1))
    #ax.set_yticklabels(np.arange(1, n+1, 1))

    # Minor ticks
    ax.set_xticks(np.arange(-.5, n, 1), minor=True)
    ax.set_yticks(np.arange(-.5, n, 1), minor=True)

    # Gridlines based on minor ticks
    ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

    # Remove minor ticks
    ax.tick_params(which='both', bottom=False, left=False)
    ax.tick_params(which='major', bottom=False, left=False)
    fig.tight_layout()
  return

In [ ]:
plot_bars_and_stripes(4)

The function`bars_and_stripes(n)` generates valid images of size $n^2$, which requires $N = 2*n$ qubits.

For a MWE we will use `n = 2` to fit the Bars and Stripes distribution, where the trained model should produce valid images on $2 \times 2$ pixels.  The QCBM needs $N = 4$ qubits.  

The other model parameters are (arbitrarily) chosen as $L = 2$ (layers) and the QCBM is sampled with $n_s = 128$ shots.  The optimizer used is Adam and the learning rate is set (again arbitrarily) to $\alpha = 0.5$.

In [ ]:
target = bars_and_stripes(2)
target = target/np.sum(target)


In [ ]:
n_wires = 4
n_layers = 2
n_shots = 128
my_qcbm = QCBM(wires=n_wires,layers=n_layers,shots=n_shots,backend='sim')

The training starts from a random unitary --the QCBM parameterized ansatz evaluated at a random choice of parameters.  

This MWE starts at the all zero vector $\theta_0 = [0, 0, \dots, 0]$.  The number of parameters needed is determined by the layer template you are using, and if the final layer of rotations is included or not.

In [ ]:
thetas = np.zeros(n_layers*n_wires) #When building the BasicEntangler layout on n_wires, each layer has (n_wires) parameters

In [ ]:
trained_model,loss_curve,all_params = my_qcbm.fit(thetas,qml.AdamOptimizer,kl_div,target,nsteps=64,alpha=0.5)

In [ ]:
print('here are the final trained parameters')
print(trained_model)

In [ ]:
output = pd.DataFrame(my_qcbm.pdf(thetas))
output.columns = ['initial']
output['fitted'] = my_qcbm.pdf(trained_model)
output['target'] = target
f, ax = plt.subplots(figsize = (10,6))
output.plot(kind='bar',legend=True,ax=ax,rot=0)

In [ ]:
print('here is the loss curve during training')
ax=sns.lineplot(loss_curve.values(),legend=False)
ax.set(yscale='log')

In [ ]:
print('here are all the parameter updates at each step of the training')
all_params

### (Optional Tests to Explore):

1. Change the number of shots used to sample from the QCBM (Caution: don't just change the `shots`, you also need to update the `device` properties)
2. Add functionality to return a valid bar or stripe from the trained QCBM, not just the `pdf()`
3. There may be large jumps in the loss during minimization, is that only dependent on the number of shots, or is there any interesting changes in the parameters during training?  

# Section 2: Training a Generative model on continuous valued data

This is a modification of how the target distribution is constructed.  In the previous section, the Bars and Stripes dataset was defined with a discrete set of valid outputs (images) to generate.

The target is constructed from continuous-valued data distribution.  

To map continuous-valued data to a discrete distribution, the values are binned and binary labels are assigned to each bin.



In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
n_samples=2000000
mu0 = [1,2.5]
cov0 = [[0.5, 0], [0, 0.5]]

mu1 = [-4,-5]
cov1 = [[2, .1], [1, 0.4]]


mu2 = [-2,-1.3]
cov2 = [[-1, .7], [.1, 0.4]]

X0 = np.random.multivariate_normal(mu0, cov0, n_samples)
X1 = np.random.multivariate_normal(mu1, cov1, n_samples)
X2 = np.random.multivariate_normal(mu2, cov2, n_samples)
X = np.concatenate([X0, X1,X2])





In [ ]:
H,x_edges,y_edges = np.histogram2d(X[:,0], X[:,1], bins=64,density=True)
#  the number of bins per dimension requires log(bins) qubits
#  with bins=64 we would need 6 qubits (12 total) to map the entire pdf
#  with bins = 128 we would need 7 qubits (14 total) to represent the entire pdf

In [ ]:
plt.imshow(H)

For this MWE the target distribution will be discretized with few qubits ($N = 3$) per dimension (to reduce the total number of needed qubits).

In [ ]:
n = 3 # qubits per dimension

H,x_edges,y_edges = np.histogram2d(X[:,0], X[:,1], bins=2**n,density=True)

y0_labels = [np.binary_repr(x,n) for x in range(2**n)]
y1_labels = [np.binary_repr(x,n) for x in range(2**n)]

target = {}
for index in np.ndindex((2**n,2**n)):
    target[y0_labels[index[0]]+y1_labels[index[1]]]=H.T[index]

QcountSorted = collections.OrderedDict(sorted(target.items()))
P_target = [x for x in QcountSorted.values()]
P_target = np.asarray(P_target)/np.sum(P_target)

In [ ]:
plt.matshow(H+1e-10, norm=LogNorm(vmin=H.min(), vmax=H.max()))

In [ ]:
plt.plot(P_target)

In [ ]:
n_wires = 2*n
n_layers = 8
n_shots = 5000
my_qcbm = QCBM(wires=n_wires,layers=n_layers,shots=n_shots,backend='sim')

In [ ]:
thetas = (np.pi/2.)*np.random.random(n_layers*n_wires+2*n_wires*my_qcbm.add_final_layer)

The call to `.fit()` is the same as the previous section, however with a larger QCBM and the need for more decimal places in the target, the number of shots is increased to 5000

In [ ]:
trained_model,loss_curve,all_params = my_qcbm.fit(thetas,qml.AdamOptimizer,kl_div,P_target,nsteps=64,alpha=0.5)

The first example used a large learning rate $\alpha = 0.5$, and during training there are some clear indications this may be too large.

In [ ]:
ax=sns.lineplot(loss_curve.values(),legend=False)
ax.set(yscale='log')

In [ ]:
trained_model,loss_curve,all_params = my_qcbm.fit(thetas,qml.AdamOptimizer,kl_div,P_target,nsteps=64,alpha=0.05)

Reducing the learning rate improves the behavior of the gradient descent optimizer.

In [ ]:
ax=sns.lineplot(loss_curve.values(),legend=False)
ax.set(yscale='log')

When testing different training parameters, try adding the final layer of rotations.

Remember, in `build_circuit(...)` there is an additional line of code that is needed to ensure that the parameters have the correct format for the way this layer is currently implemented.

If adding the final layer of rotations, then the QCBM has an additional $2n$ parameters.

In [ ]:
n_shots = 64000
my_qcbm = QCBM(wires=n_wires,layers=n_layers,shots=n_shots,backend='sim',final_layer=True)
thetas = (np.pi/2.)*np.random.random(n_layers*n_wires+2*n_wires*my_qcbm.add_final_layer)-np.pi/4.

In [ ]:
trained_model,loss_curve,all_params = my_qcbm.fit(thetas,qml.AdamOptimizer,kl_div,P_target,nsteps=64,alpha=0.05)

Furthermore, increasing the number of shots also improves performance.

In [ ]:
ax=sns.lineplot(loss_curve.values(),legend=False)
ax.set(yscale='log')

Comparing the fitted distribution to the target, there is still room for improvement.

In [ ]:
output = pd.DataFrame(my_qcbm.pdf(thetas))
output.columns = ['initial']
output['fitted'] = my_qcbm.pdf(trained_model)
output['target'] = P_target
f, ax = plt.subplots(figsize = (16,6))
ax.set(yscale='log')
output.plot(kind='bar',legend=True,ax=ax,rot=0)

In [ ]:
H_pred = my_qcbm.pdf(trained_model).reshape((2**n,2**n))

In [ ]:
plt.matshow(H_pred+1e-10, norm=LogNorm(vmin=H.min(), vmax=H.max()))

## Optional (Next steps)

Going beyond this MWE, how can the fitting be improved?  This is a good example of the need for hyperparameter optimization, as well as determining the optimal ansatz design.

1) Increasing the depth, shots, or number of optimization steps
\
2) decreasing the learning rate
\
3) Changing the cost function
\
4) Adjusting the mesh spacing used to discretize the target

## Section 3: Training a generative model with input

So far, the QCBMs trained in this network take a fixed input state and return samples drawn from a fixed distribution.  But these samples are returned by uniform sampling.
What if we want to provide partial information or an input feature to the model?  For example, one use case for generative models is *denoising*.  A trained model is used to extract a "true signal" from a corrupted input.  For this MWE we are going to return a valid image from the Bars and Stripes dataset that has been corrupted from some noise.

To explore this, need a way to encode input vectors $x$ into the model (in `build_circuit(...)`), and the training worfklow needs to be modified (in `fit(...)`).  Before making changes, let's make a copy of the `QCBM` class and make modify it.

First, in `build_circuit(...)` let's adapt the generator design used in Style-GAN models [2].  This was used with adversarial training in [3], we're looking at non-adversarial training (no discriminator).  The main component that needs to be added, is the ability to encode latent representations of input features, periodically in the unitary circuit.

A full Style-GAN model will first use a deep neural network to define that latent representation and the style-generator encodes these latent features into the gnereator. In this MWE the latent space is assumed to be the one spanned by one-hot encoding vectors. Given an image (or corrupted image) of $N$ pixels, each corrupted pixed is encoded as a length $N$ zero vectorand only the entry for pixel [i] is non-zero,
$$ v[i] = p[i] $$
the $i$-th entry of the vector is equal to the pixel value.

Recall from the description of the Bars and Stripes dataset above, the pixels are mapped to binary variables $\lbrace 0, 1\rbrace$.  An ideal image (without any corruption) consists of $N$ binary variables, which are now individually encoded as length-$N$ vectors.  

Example:  the $i=2$ pixel in a $4$ pixel image is white $p[i]=1$.  The vector representation of this pixel only is $[0, 0, 1, 0]$.

Example:   the $i=0$ pixel in a $4$ pixel image has been corrupted, and is $0.5$.  The vector representation of this pixel only is $[0, 0.5, 0, 0]$.

References:
\
[2] Karras, Tero, Samuli Laine, and Timo Aila. "A style-based generator architecture for generative adversarial networks." Proceedings of the IEEE/CVF conference on computer vision and pattern recognition. 2019. [StyleGAN CVPR paper](https://openaccess.thecvf.com/content_CVPR_2019/html/Karras_A_Style-Based_Generator_Architecture_for_Generative_Adversarial_Networks_CVPR_2019_paper.html)
\
[3]Bravo-Prieto, Carlos, et al. "Style-based quantum generative adversarial networks for Monte Carlo events." Quantum 6 (2022): 777. [StyleQGAN](https://quantum-journal.org/papers/q-2022-08-17-777/)





In [ ]:
def plot_noisy_bars_and_stripes(n,p):
  ''' plot noisy bars and stripes images of size n x n
  where a pixel has a probability (p) of being corrupted'''
  valid_images = np.where(bars_and_stripes(n)>0)[0]


  w = len(valid_images)//4
  h = 4
  fig, axs = plt.subplots(nrows=h, ncols=w,figsize=(10,10))

  for idx, ax in enumerate(axs.flat):
    noiseless_image = [int(x) for x in np.binary_repr(valid_images[idx],width=n**2)]
    # TODO: how are the images corrupted:  Gaussian noise?  are random pixels erased?
    # ADD CODE HERE
    # define a random vector that corrupts a noiseless valid image
    noisy_image = ...
    ax.imshow(np.array(noisy_image).reshape((n,n)).numpy(),cmap='coolwarm')
    # Major ticks
    ax.set_xticks(np.arange(0, n, 1))
    ax.set_yticks(np.arange(0, n, 1))

    # Minor ticks
    ax.set_xticks(np.arange(-.5, n, 1), minor=True)
    ax.set_yticks(np.arange(-.5, n, 1), minor=True)

    # Gridlines based on minor ticks
    ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

    # Remove minor ticks
    ax.tick_params(which='both', bottom=False, left=False)
    ax.tick_params(which='major', bottom=False, left=False)
    fig.tight_layout()
  return

In [ ]:
plot_noisy_bars_and_stripes(3,0.001)

The first approach to denoising is to use training data generated by $m$ noisy observations of a **known** image (i.e. we have access the ground truth).  So training a denoising model is a supervised learning task.

Before the `fit(...)` function is modified we need to generate the training data.  For this MWE we only use one randomly chosen image from the Bars and Stripes dataset, then, depending on our noise model, we generate $m$ noisy versions.

The training dataset will be composed as $X = \lbrace \widetilde{y}_i,y\rbrace$.  Each corrupted (noisy) example ($\widetilde{y}_i$) is labeled by the known ground truth $y$.

In [ ]:
def generate_multiple_noisy_samples(n,m,p):
  ''' randomly generate an image from the Bar and Stripe dataset with pixel noise
  randomly corrupted with noise of strength p'''
  valid_images = np.where(bars_and_stripes(n)>0)[0]
  random_index = np.random.choice(len(valid_images))
  noiseless_image = np.array([int(x) for x in np.binary_repr(valid_images[random_index],width=n**2)])
  output = []
  for idx in range(m):
    # TODO -- add code to implement a particular noise model and corrupt the image
    # Ensure that unique corrupted image can be produced
    noise = ...
    noisy_image = ...
    output.append(noisy_image.numpy())
  output.append(noiseless_image) # the ground truth is appended at the end
  return np.array(output)


In [ ]:
def build_noisy_clean_pairs(noisy_samples):
  X = []
  y = []
  for idx in range(noisy_samples.shape[0]-1):
    X.append(noisy_samples[idx]) #corrupted image is the feature
    y.append(noisy_samples[-1]) #ground truth is the label
  return np.asarray(X),np.asarray(y)

In [ ]:
def build_training_pairs(noisy_samples):
  X = []
  y = []
  for subset in it.combinations(list(range(noisy_samples.shape[0])), 2):
    X.append(noisy_samples[subset[0]])
    y.append(noisy_samples[subset[1]])
  return np.asarray(X),np.asarray(y)


In [ ]:
# verify that the training pairs are correctly formatted

# The `StyleQCBM` class

**This Section does not have a MWE -- the QCBM preparation may work, but it may not implement the correct feature encoding**

This is a copy (literaly copy-paste) of the `QCBM` class above, this is the code that will be adapted to implement and train a denoising model.

Modifications needed:

1) `build_circuit(...)` now takes as a keyword `x` an input feature.  The function must be updated to ensure that these feature is used during the state preparation.

2) the function used to extract output, either `pdf(...)` or `spin_vector(...)` must also take `x` as a feature vector

3) the cost function is now a method of the class, an example of the mean square error loss is given, and it takes `x` (feature) and `y` (label)

4) the `fit(...)` function must be updated to implement supervised learning.  A skeleton is given, however it assumes that the modifications 1--3 have been implemented

In [ ]:
class StyleQCBM:
    '''
    Parent class for constructing
    a style-based (inspired) Quantum Circuit Born Machine, that uses PennyLane
    Inputs:
    wires -- number of qubits to build in the circuit
    shots -- number of shots to take from the prepared state
    layers -- how many layers of a template to add to the circuit (using a template)
    template -- which predefined template to use:
      'basic_template' = BasicEntanglerLayers,
      'strong_tempalte' = StronglyEntanglingLayers,
      'two_design_template' = SimplifiedTwoDesign,
      'user_defined_template' = user defined function
    backend -- 'sim' or 'noisy_sim' for simulator or noisy simulator
    **kwargs -- additional parameters for the class
    '''


    def __init__(self, wires=1,shots=1024,layers=1,backend='sim',**kwargs):
        self.dev_wires = [np.array(idx, requires_grad=True) for idx in range(wires)] # the individual device wires
        self.wires=wires # number of qubits in the circuit
        self.rotation = kwargs.get('rotation',None) #option for the BasicEntanglingLayer template which uses single axis rotations as the parameterized gate
        self.layers = layers # number of layers of template to add to the circuit
        self.layout = kwargs.get('layout','basic_template') # which template to use
        self.add_final_layer=kwargs.get('final_layer',False) # option for adding a final layer of rotations to the circuit
        self.backend=backend # options: 'sim' or 'noisy_sim'
        self.shots = shots # number of shots to sample from the final state
        self.n_iter_no_change=kwargs.get('wait_time',5) # hyperparameter for training that triggers early stopping
        self.tol = kwargs.get('tol',1e-5) # hyper parameter for training that triggers early stopping
        self.pBF=kwargs.get('pBF',None)  # option for noisy simulation controlling the strength of BitFlip errors prior to measurement
        self.pDP=kwargs.get('pDP',None) # option for noisy simulation controlling the strength of Depolarizing noise channel
        self.pAD=kwargs.get('pAD',None) # option for noisy simulation controlling the strength of Amplitude Damping noise channel
        self.pCE=kwargs.get('pCE',None) # option for noisy simulation controlling the strength of coherent errors
        self.prob_vec = True
        if self.backend=='sim':
            self.device = qml.device("default.qubit", \
                                     wires=self.dev_wires,shots=shots)
        elif self.backend=='noisy_sim':
            if (self.pDP!=None) or (self.pBF!=None)or (self.pAD!=None):
                ##print('you need a noisy device')
                self.device = qml.device('default.mixed',\
                                    wires = self.dev_wires,shots=shots)
        self.rng=np.random.default_rng(2024)

    def write_metadata(self):
      """Option: Add this functionality"""
      """ Add code here to initialize a directory to store output files"""
      """ Example: trained model parameters, loss curves, information about hyperparameters used"""
      raise NotImplementedError

    def build_circuit(self,params,x=None):
      """ Option: Add some code to check that the correct number of parameters
      have been passed to build the circuit """
      # # #
      # Here
      # # #
      if self.add_final_layer:
        unitary_params = params[:-self.wires*2]
        final_params = params[-self.wires*2:]
      else:
        unitary_params = params
        final_params = None
      """ TODO Modify build_circuit construction  """
      # # #
      # Add/modify the code here to encode the feature (x)
      # the feature (x) is assumed to be the corrupted image
      # need to add code to 1) convert each pixel to a one-hot encoded vector
      # and 2) encode these pixel vectors into the state being prepared by the circuit
      # # #
      qml.BasisState(np.zeros(self.wires), wires=self.dev_wires)
      if self.layout=='basic_template':
        shape = BasicEntanglerLayers.shape(n_layers=self.layers,\
                                                      n_wires=self.wires)
        unitary_params=np.asarray(unitary_params).reshape(shape)
        BasicEntanglerLayers(unitary_params,rotation=self.rotation,\
                                                        wires = self.dev_wires)
      elif self.layout=='strong_template':
        shape = StronglyEntanglingLayers.shape(n_layers=self.layers,\
                                                        n_wires=self.wires)
        unitary_params=np.asarray(unitary_params).reshape(shape)
        StronglyEntanglingLayers(unitary_params, wires = self.dev_wires)
      elif self.layout=='two_design_template':
        shape = SimplifiedTwoDesign.shape(n_layers=self.layers, \
                                                        n_wires=self.wires)
        unitary_params=np.asarray(unitary_params).reshape(shape)
        SimplifiedTwoDesign(unitary_params, wires = self.dev_wires)
      elif self.layout=='user_defined_template':
        # ADD YOUR CODE HERE
        # # #
        # write a function that can be passed to qml.template or use other
        # functionalitiy such as qml.broadcast to build your own parameterized
        # circuit design
        # # #
        raise NotImplementedError
      else:
        raise ValueError('Please choose a valid layout')
      if final_params is not None:
        # ADD YOUR CODE HERE
        # Ensure that final_params has the correct shape
        final_params = final_params.reshape((2,-1))
        qml.broadcast(unitary=qml.RY, pattern="single", \
                            wires=self.dev_wires, parameters=final_params[0])
        qml.broadcast(unitary=qml.RX, pattern="single", \
                      wires=self.dev_wires, parameters=final_params[1])

      if self.prob_vec:
          if self.pBF!=None:
              for idx in self.dev_wires:
                  qml.BitFlip(self.pBF, wires=self.dev_wires[int(idx)])
              return qml.probs(wires=self.dev_wires)
          return qml.probs(wires=self.dev_wires)
      else:
          return np.asarray([qml.expval(qml.PauliZ(i)) for i in self.dev_wires])

    def draw_circuit(self,*args, **kwds):
        raise NotImplementedError

    def pdf(self,params,x=None):
        '''
        pennylane returns the probabilities in an OrderedDict - no need for sorting the values
        '''
        self.prob_vec=True
        if x is None:
            x = np.zeros(self.wires)
        qnode_ = qml.QNode(self.build_circuit, self.device)
        Q = qnode_(params,x)
        return Q


    def prediction(self,params,x=None):
        '''
        ADD This functionality:  given x and a parameter vector, return the predicted
        image from the model
        '''
        # ADD CODE HERE
        raise NotImplementedError
        return

    def spin_vector(self,params,x=None):
      """ ADD CODE HERE """
      # Add code so that the output of the QCBM is a vector of [<Z0>, <Z1>,...]
      raise NotImplementedError
      return

    def sample(self,*args,**kwds):
      """ ADD CODE HERE"""
      # Add code to prepare the pdf and draw N samples
      # return samples as bitstrings s
      raise NotImplementedError

    def sigmoid(self,x):
        return 1/(1+np.exp(-x))

    def cost_function(self,params, x, y):
      """Cost function to be minimized.

      Args:
          params (array[float]): array of parameters
          x (array[float]): 2-d array of input vectors
          y (array[float]): 1-d array of targets
          state_labels (array[float]): array of state representations for labels

      Returns:
          float: loss value to be minimized
      """

      # This function requires the prediction function above
      pred = self.prediction(params,x)
      loss =np.mean((y - pred) ** 2)
      return loss

    def fit(self,startval,X,y,opt_method,nsteps=10,alpha=0.05):
      """ ADD CODE HERE """
      # Add code to handle the case that an argument is missing

      opt = opt_method(stepsize=alpha)

      params = startval.copy()

      loss = {}
      # if cost_function is implementable, this returns the mean cost evaluated
      # at the start of training
      loss[0]=np.mean(np.sum([self.cost_function(params,X[i],y[i]) for i in range(len(X))]))
      print('initial loss: ',loss[0])

      # Option:  if you have a function to make predictions coded up
      # preds = self.make_predictions(params,X)

      stored_params = {}
      stored_params[0]=params.copy()

      for i in range(nsteps):
          # Update the circuit parameters -- this implements supervised learning
          # using online learning: the parameters are updated based on a gradient
          # step defined by the loss evaluated at a single sample-label pair
          # OPTIONAL:
          # Modify this to use all samples, or batches of samples
          for j in range(len(X)):
            params = opt.step(lambda v: self.cost_function(v, X[j], y[j]), params)
            self.coefs_ = params.copy()
            # Option: Does this print out too much during training?  add a verbose keyword to toggle that on/off
          if i%2==0:
            _loss = np.mean(np.sum([self.cost_function(params,X[i],y[i]) for i in range(len(X))]))
            print(i+1,_loss)
          loss[i+1]=_loss.copy()
          stored_params[i+1]=params.copy()
      return params,loss,stored_params



In [ ]:
n_wires = 4
n_layers = 2
n_shots = 1024
my_qcbm = StyleQCBM(wires=n_wires,layers=n_layers,shots=n_shots,backend='sim',add_final_layer=False)

In [ ]:
thetas = (np.pi)*np.random.random(n_layers*n_wires+2*n_wires*my_qcbm.add_final_layer)-np.pi/2.

Visually inspect the circuit -- is it encoding the `x` feature as you expect?

In [ ]:
temp_qnode = qml.QNode(my_qcbm.build_circuit, my_qcbm.device)
qml.draw_mpl(temp_qnode,decimals=3,expansion_strategy='device')(thetas,x=np.array([0,1,0.5,1]))


In [ ]:
my_qcbm.prediction(thetas,x=np.array([1,0,0.5,1])  )

In [ ]:
noisy_samples = generate_multiple_noisy_samples(2,15,0.5)
X,y = build_noisy_clean_pairs(noisy_samples)

Again, look at the generate dataset, verify it is properly created

In [ ]:
np.round(X,2)

In [ ]:
trained_model,loss_curve,all_params = my_qcbm.fit(thetas,X,y,qml.AdamOptimizer,nsteps=20,alpha=0.05)

In [ ]:
g=sns.lineplot(loss_curve.values(),legend=False)
g.set(yscale='log')

## Optional:  Compare images

1) Import `metrics` from `scikit-learn` and use to compare similarities between predictions and the known ground truth image.

2)  Use `matplotlib` to generate plots comparing the noisy, generated output, and the ground truth

In [ ]:
trained_model_predictions = []
for idx in range(len(X)):
  trained_model_predictions.append(my_qcbm.prediction(trained_model,X[idx]))
trained_model_predictions.append(y[0])

In [ ]:
from sklearn import metrics

In [ ]:
M = metrics.pairwise.cosine_similarity(trained_model_predictions)

In [ ]:
plt.imshow(M)

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=3,figsize=(10,10))
n = 2
idx = 2
axs[0].imshow(np.array(X[idx]).reshape((n,n)).numpy(),cmap='coolwarm',vmin=0,vmax=1)
for i in range(2):
    for j in range(2):
        text = axs[0].text(j, i, np.round(np.array(X[idx]).reshape((n,n)).numpy()[i, j],3),
                       ha="center", va="center", color="w")
axs[1].imshow(np.array(trained_model_predictions[idx]).reshape((n,n)).numpy(),cmap='coolwarm',vmin=0,vmax=1)
for i in range(2):
    for j in range(2):
        text = axs[1].text(j, i, np.round(np.array(trained_model_predictions[idx]).reshape((n,n)).numpy()[i, j],3),
                       ha="center", va="center", color="w")
axs[2].imshow(np.array(y[idx]).reshape((n,n)).numpy(),cmap='coolwarm',vmin=0,vmax=1)
for i in range(2):
    for j in range(2):
        text = axs[2].text(j, i, np.round(np.array(y[idx]).reshape((n,n)).numpy()[i, j],3),
                       ha="center", va="center", color="w")

for ax in axs.flatten():
  # Major ticks
  ax.set_xticks(np.arange(0, n, 1))
  ax.set_yticks(np.arange(0, n, 1))

  # Labels for major ticks
  #ax.set_xticklabels(np.arange(1, n+1, 1))
  #ax.set_yticklabels(np.arange(1, n+1, 1))

  # Minor ticks
  ax.set_xticks(np.arange(-.5, n, 1), minor=True)
  ax.set_yticks(np.arange(-.5, n, 1), minor=True)

  # Gridlines based on minor ticks
  ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

  # Remove minor ticks
  ax.tick_params(which='both', bottom=False, left=False)
  ax.tick_params(which='major', bottom=False, left=False)
fig.tight_layout()


## Optional (Next Steps)

1. How few pairs of (clean, noisy) images can be used to train the model?

2. What is the maximum level of noise that the model can learn under?

3. Try training a model with multiple unique images

4. This model was trained under the assumption that the images were corrupted, but the qubits and circuit are noiseless.  How does adding quantum noise affect this model?  How noisy can the `StyleQCBM` be, before it no longer can be trained as an image denoiser?